In [1]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import os
import json
from time import strftime,gmtime
from opencc import OpenCC
from sklearn.utils import shuffle
import numpy as np
import random
print(torch.cuda.is_available())

True


In [2]:
class DRCDdataset():
  def __init__(self,train_path=None , test_path = None ,dev_path = None):
    
    if train_path != None:
      self.train = self.get_data(train_path)
    if test_path != None:
      self.test = self.get_data(test_path)
    if dev_path != None:
      self.dev = self.get_data(dev_path)

  def get_data(self,path):
    input_file = open(path)
    ds = json.load(input_file)
    datas = []
    for i in range(len(ds['data'])):
      for j in ds['data'][i]['paragraphs']:
        context = j['context']
        for qa in j['qas']:
            question = qa['question']
            ans = set([ a['text'] for a in  qa['answers']])
            datas.append([context,question,ans])
       # question = [q['question'] for q in j['qas']]
       # ans = [ a['answers'][0]['text'] for a in j['qas'] ]
        #ans = [a['answers'][0]['answer_start'] for a in j['qas']]
        #anse = [a['answers'][0]['answer_start']+len(a['answers'][0]['text'])-1 for a in j['qas']]
        #for k in range(len(question)):
         # datas.append([context,question[k],ans[k]])
          #datas.append([context,question[k],ans[k],anse[k]])
    return datas


In [3]:
"""
class bertDRCD(nn.Module):
    def __init__(self,model_type):
        super(bertDRCD,self).__init__()

        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type,config = config)
  
        self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        
    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): 
      # hidden (batch,seq_len,hidden)
      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]
      s = torch.sum(hidden*self.s_vector,-1) # s now is (batch,seq_len)
      M = token_type_ids.clone().float().to(hidden.device).detach()
      M[M != 1] = float('-inf')
      s = s+M 
      #s = torch.softmax(s,dim=-1)
      e = torch.sum(hidden*self.e_vector,-1)
      e = e + M
      #e = torch.softmax(e,dim=-1)
      return s,e
"""

"\nclass bertDRCD(nn.Module):\n    def __init__(self,model_type):\n        super(bertDRCD,self).__init__()\n\n        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)\n        self.bert_model = BertModel.from_pretrained(model_type,config = config)\n  \n        self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) \n        self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) \n        \n    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): \n      # hidden (batch,seq_len,hidden)\n      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]\n      s = torch.sum(hidden*self.s_vector,-1) # s now is (batch,seq_len)\n      M = token_type_ids.clone().float().to(hidden.device).detach()\n      M[M != 1] = float('-inf')\n      s = s+M \n      #s = torch.softmax(s,dim=-1)\n      e = torch.sum(hidden*self.e_vector,-1)\n      e = e + M\n      #e = torc

In [4]:
class bertDRCD(nn.Module):
    def __init__(self,model_type):
        super(bertDRCD,self).__init__()

        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type,config = config)
  
        #self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        #self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 

        self.s_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
        ) 
        self.e_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
        )
        #self.start()

    def start(self):
           
      nn.init.xavier_uniform_(self.s_decoder[0].weight)
      nn.init.constant_(self.s_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.s_decoder[3].weight)
      nn.init.constant_(self.s_decoder[3].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[0].weight)
      nn.init.constant_(self.e_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[3].weight)
      nn.init.constant_(self.e_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): 
      # hidden (batch,seq_len,hidden)
      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]
      s = self.s_decoder(hidden).squeeze()
      M = token_type_ids.clone().float().to(hidden.device).detach()
      M[M != 1] = float('-inf')
      s = s+M 
      #s = torch.softmax(s,dim=-1)
      e = self.e_decoder(hidden).squeeze()
      e = e + M
      #e = torch.softmax(e,dim=-1)
      return s,e

In [13]:
class DRCD():
    def __init__(self,model_path,model_type,language): 
        # device
        self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
        # model & tokenizer
        # model_type = 'hfl/chinese-roberta-wwm-ext'
        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.tokenizer = BertTokenizer.from_pretrained(model_type)
        self.model = bertDRCD(model_type).to(self.device)
        self.model.load_state_dict(torch.load(model_path)) 
        self.language = language
        # 繁簡轉換
        self.c2tw = OpenCC('s2t') # china to tw
        self.tw2c = OpenCC('t2s') # tw to china

    def process(self,content,question):
        
        content,question = self.clean_str(content,question)
        
        
        with torch.no_grad():
            
            token_tensor = self.tokenizer.encode_plus(str(question),str(content),max_length=512,truncation=True,pad_to_max_length=True)
            token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
            segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
            mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
            answer_start,answer_end = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment) 
            answer_start = torch.softmax(answer_start.squeeze(),dim=-1)
            answer_end =  torch.softmax(answer_end.squeeze(),dim=-1)
            # get ans
            tokens = self.tokenizer.convert_ids_to_tokens(token.squeeze())
            #answer_start = answer_start.argmax(1)
            #answer_end = answer_end.argmax(1)

            answer_start = torch.topk(answer_start, 3)[1].tolist()
            answer_end = torch.topk(answer_end,3)[1].tolist()
            
       
            start_idx,end_idx = -1,-1
           # print(answer_start,answer_end)
            find_ans = False
            for s in range(0,len(answer_start)):
                for e in range(0,len(answer_end)):
                    if (answer_start[s] < answer_end[e]) and (answer_end[e]-answer_start[s]) < 20:
                        start_idx = answer_start[s]
                        end_idx = answer_end[e]
                        find_ans = True
                        break
                if find_ans:
                    break
            if start_idx < 0:
                return "No answer",([],[])
           # print(start_idx,end_idx)

            
            #print(answer_start,answer_end)
            #if answer_start > answer_end :
            #    return '抱歉 這題有點難耶'
            
            #print(answer_start,answer_end)
            
            answer = ''.join(tokens[start_idx:end_idx+1])
            answer = answer.replace('##','')
            if self.language == 'china':
                answer = self.c2tw.convert(answer)
            

            
        
        return answer, (answer_start,answer_end)
    
    def clean_str(self,content,question):
        content = content.replace(' ','')
        question = question.replace(' ','')
        if self.language == 'china':
            content = self.tw2c.convert(content)
            question = self.tw2c.convert(question)

        return content,question

In [14]:
modelname = 'bertDRCD_0812_0034.pt'
myfriend = DRCD(f'saved_models/{modelname}','hfl/chinese-roberta-wwm-ext','tw')

In [15]:
ds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')

In [8]:
#ds.dev[:10]
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
content = '現在突厥語西北語支名為欽察語支，有些突厥部落是他們的後人：西伯利亞韃靼人、諾蓋人、哈薩克人、克里米亞韃靼人、卡拉恰伊人、庫梅克人、卡拉伊姆人、卡拉卡爾帕克族、烏茲別克人、吉爾吉斯人、伏爾加韃靼人、巴什基爾人。克里米亞韃靼人和哈薩克人的地理位置、生活方式、語言，保留了最多欽察人特色。哈薩克人與吉爾吉斯人有欽察部落，克里米亞有一條村名為欽察，欽察一名也出現在烏古斯可汗傳說的史詩中。蒙古有一部落哈剌赤，據說與土土哈的父親班都察有關，他全部投降，這部落善於製做黑馬乳，因此命名為哈剌赤。匈牙利現在仍然有一部落自稱欽察的後人，人數大約20萬。他們是當年隨忽炭汗前往匈牙利的欽察人之後。四川西昌的部分俞姓被認為是欽察人土土哈之後，俞是玉里伯里的對音，但人數不詳。欽察也是回族來源之一。在準噶爾汗國時代還有欽察人生活在費爾干納。在近歐的一些欽察人，於11世紀左右時由於曾與喬治亞人聯合來對抗伊斯蘭勢力的入侵，在喬治亞人的影響下，有相當數量的人口改信基督教。這些人在喬治亞的建國者大衛四世的要求下受浸，而大衛四世本身亦迎娶了欽察汗Otrok的女兒古蘭杜赫特為妻。從1120年開始，欽察汗國以基督教為國教，並有自己的國家教會和神職人員。不過，到了12世紀至13世紀期間，伊斯蘭勢力已在花剌子模國北部欽察內深深地紮根了。'
#tokenizer(content)

In [9]:
myfriend.process(ds.dev[0][0],ds.dev[0][1])

('歐洲', ([26, 38, 25], [27, 39, 26]))

# EM

In [10]:
# bertDRCD_0812_0034.pt 1e-5 hfl/chinese-roberta-wwm-ext  bs:4 ep:5 #s_decoder+e_decoder
# 0.7225880332092757
# bertDRCD_0812_0034.pt 1e-5 hfl/chinese-roberta-wwm-ext  bs:4 ep:5 #s_decoder+e_decoder tok-3:
# 3493 2728 0.7809905525336387

In [16]:
all_num = 0
match_num =0

for c,q,a in ds.test:
    all_num = all_num+1
    pred,place = myfriend.process(c,q)
    if pred in a:
        match_num = match_num +1
    else: 
        print(place)
        print(pred,a)
        print('===============')

print(all_num,match_num,match_num/all_num)

([150, 149, 153], [153, 152, 151])
八篇書》 {'《八篇書》', '八篇書'}
([361, 46, 362], [362, 47, 361])
俗語 {'梵語'}
([54, 33, 49], [55, 52, 34])
古典 {'吠陀梵語'}
([47, 298, 172], [49, 300, 174])
天城體 {'悉曇體'}
([29, 49, 275], [31, 34, 50])
西莒島 {'西莒島青帆村'}
([49, 81, 88], [53, 46, 51])
長樂鶴上人 {'不願被俘而自沉於海', '自沉於海'}
([90, 196, 270], [91, 197, 271])
媽祖 {'馬祖'}
([49, 53, 73], [51, 54, 50])
北竿島 {'北竿島的壁山', '壁山'}
([38, 127, 123], [131, 143, 40])
花崗岩 {'侵蝕作用'}
([392, 410, 409], [393, 413, 392])
颱風 {'1060公釐'}
([107, 99, 61], [110, 106, 71])
中午12點 {'西元前4713年1月1日中午12點'}
([116, 165, 214], [175, 168, 216])
1月2日12:00的24小時 {'0', '2450084'}
([240, 245, 236], [247, 248, 245])
一盤中戰至6比6 {'當雙方在一盤中戰至6比6時', '當雙方在一盤中戰至6比6'}
([], [])
No answer {'決勝局必定是單數局', '因為決勝局必定是單數局'}
([51, 46, 49], [54, 52, 389])
單打比賽 {'大滿貫男子單打比賽'}
([345, 350, 347], [358, 341, 352])
一局特殊的決勝局結束一盤比賽 {'採用一局特殊的決勝局', '在雙方平局局數足夠高時將不再延長比賽而採用一局特殊的決勝局結束一盤比賽'}
([103, 77, 76], [103, 104, 107])
7局 {'每9局', '每9局比賽'}
([209, 220, 254], [263, 222, 245])
健全人或需要輪椅移動的殘疾人 {'可在球反彈2次後再擊球',

([237, 51, 235], [238, 52, 237])
第六 {'六'}
([151, 235, 156], [238, 152, 231])
北韓 {'「韓」', '南·韓國'}
([240, 245, 253], [257, 243, 267])
韓國高鐵將高速公路形成的「一日生活圈 {'高鐵', '韓國高鐵將高速公路'}
([69, 57, 59], [61, 82, 73])
古朝鮮准王建立衛滿朝鮮。西漢 {'古朝鮮准王', '古朝鮮'}
([153, 91, 94], [95, 138, 153])
大米和薯類 {'糧食', '其他糧食'}
([85, 79, 82], [86, 83, 80])
2001年 {'1996年'}
([181, 179, 157], [182, 181, 178])
兩年 {'不足兩年'}
([206, 196, 214], [207, 212, 209])
冷戰 {'冷戰格局'}
([296, 237, 253], [253, 255, 296])
陸軍現役軍人有49.5萬人，裝備有2000 {'20'}
([137, 211, 181], [215, 142, 186])
2012年4月17日 {'2015年12月1日'}
([77, 79, 283], [80, 286, 79])
駐韓美軍 {'駐越美軍'}
([370, 320, 373], [377, 321, 374])
主語+賓語+謂語 {'主語'}
([70, 75, 88], [71, 76, 89])
元音 {'輔音'}
([131, 135, 31], [136, 138, 134])
朝鮮王朝世宗 {'世宗大王'}
([117, 108, 127], [122, 120, 113])
35.3億韓圓 {'近16倍', '16倍'}
([49, 44, 54], [50, 45, 55])
高中 {'初中'}
([185, 176, 174], [192, 191, 172])
[UNK]·ㄎㄟ·ㄨㄞ」 {'因其英文縮寫首字母', '其英文縮寫首字母為「ㄟㄙ·ㄎㄟ·ㄨㄞ」'}
([185, 170, 183], [191, 177, 188])
高麗大學校主樓 {'和信百貨公司大樓', '朴吉龍'}
([43, 37, 271], [44, 

([39, 41, 43], [137, 47, 132])
電力無法大量的儲存 {'與燃煤取暖或燃油取暖相比，它的效率比較低', '效率比較低'}
([27, 29, 28], [29, 28, 27])
信陽市 {'信陽'}
([21, 23, 22], [23, 22, 21])
信陽市 {'信陽'}
([22, 24, 23], [24, 23, 22])
信陽市 {'信陽'}
([228, 81, 252], [270, 253, 495])
西申，後遷至信陽，國都建在謝城，稱東申 {'期思陂'}
([83, 464, 79], [81, 465, 305])
申侯 {'孫叔敖'}
([89, 510, 176], [91, 95, 511])
趙光義 {'信陽長台關編鐘'}
([153, 137, 134], [138, 154, 156])
兩成 {'20%', '兩成不到'}
([505, 129, 506], [506, 508, 505])
兩波 {'38%'}
([39, 69, 138], [43, 61, 70])
古羅馬文學 {'武力', '均衡、嚴整、和諧'}
([280, 291, 153], [155, 286, 287])
《致阿提庫斯書 {'《內戰記》'}
([316, 315, 491], [317, 318, 319])
金驢 {'《羅馬十二帝王傳》'}
([49, 213, 207], [55, 51, 211])
1610年1月7日 {'戰國時代', '公元前362年'}
([83, 230, 85], [88, 235, 84])
2013年2月28日 {'2013年2月11日'}
([100, 53, 49], [102, 58, 165])
梵蒂岡 {'德語地區'}
([195, 38, 448], [196, 449, 39])
城市 {'美索不達米亞的蘇美爾人', '公元前3500年左右美索不達米亞'}
([], [])
No answer {'技術發明'}
([52, 301, 510], [53, 301, 52])
中共 {'馬來西亞'}
([41, 27, 43], [43, 42, 28])
廈門市 {'吉隆坡'}
([493, 21, 78], [22, 79, 494])
1978年 {'2016

([214, 230, 209], [238, 212, 228])
檢查自己是否放棄了那些天賦的心 {'閉門思過，檢查自己是否放棄了那些天賦的心，努力把這些心找回來', '閉門思過'}
([322, 374, 376], [325, 377, 323])
傳統思想 {'法家'}
([477, 320, 297], [322, 298, 48])
珍珠港 {'戰艦', '空中武力'}
([371, 423, 370], [352, 375, 440])
英美荷開戰 {'美國會與日本於馬利亞納群島和馬紹爾群島之間海域戰鬥', '近海戰鬥'}
([435, 436, 417], [436, 435, 437])
6500k {'6500K'}
([403, 404, 405], [404, 406, 403])
尊漢 {'尊漢思想'}
([375, 458, 463], [376, 459, 476])
阮攸 {'法國傳教士亞歷山大·羅德'}
([42, 41, 126], [43, 128, 46])
漢語 {'國語', '國語字'}
([46, 333, 366], [47, 334, 46])
漢語 {'漢文'}
([34, 33, 321], [35, 38, 78])
漢語 {'儒字'}
([265, 266, 50], [266, 265, 312])
阮攸 {'胡季犛'}
([106, 28, 30], [42, 110, 431])
缺乏標準化 {'未標準化'}
([215, 226, 216], [216, 219, 226])
xy {'XY', 'XY染色體'}
([226, 215, 216], [226, 219, 229])
xx染色體 {'XX', 'XX染色體'}
([356, 388, 51], [359, 391, 100])
雌雄同體 {'無性生殖'}
([318, 319, 305], [340, 309, 316])
有一些物種 {'同一個生物體中可能同時有雄性及雌性的生殖器官', '在同一個生物體中可能同時有雄性及雌性的生殖器官'}
([371, 372, 373], [373, 374, 372])
十億年 {'十億年前'}
([177, 275, 116], [178, 119, 173])
配子 {'接合'}
([369

([39, 47, 55], [56, 53, 46])
中世紀晚期出現的共同生活弟兄會的教士 {'共同生活弟兄會的教士'}
([159, 154, 173], [163, 157, 162])
格爾拉赫峰 {'斯洛伐克'}
([302, 300, 304], [307, 305, 63])
河本大作大佐 {'日軍河本大作大佐', '河本大作'}
([156, 139, 145], [152, 159, 151])
東北易幟 {'南京國民政府青天白日滿地紅旗', '青天白日滿地紅旗'}
([292, 293, 301], [296, 297, 299])
「革命外交 {'革命外交', '「革命外交」政策'}
([123, 124, 122], [124, 128, 123])
美日 {'美日領事機構'}
([124, 142, 127], [127, 62, 146])
瓜拿納樹 {'巴拉圭冬青'}
([307, 311, 305], [307, 185, 179])
含量少 {'少'}
([43, 54, 46], [48, 61, 50])
檸檬酸咖啡因 {'檸檬酸咖啡因療法'}
([216, 221, 302], [219, 223, 309])
費城學院 {'費城學院和研究院'}
([225, 289, 243], [227, 291, 226])
1755年 {'1779年'}
([184, 202, 197], [185, 204, 159])
靜谷 {'春耦齋'}
([213, 212, 242], [215, 214, 244])
順治帝 {'清順治帝'}
([342, 362, 353], [345, 355, 365])
三級產業 {'一級產業'}
([124, 308, 268], [128, 315, 236])
王田交流道 {'福爾摩沙高速公路'}
([114, 150, 133], [151, 115, 134])
1898年 {'1900年'}
([99, 114, 112], [117, 104, 110])
上尉特別官佐，即候差軍官，擔任教育副官 {'教育副官'}
([94, 142, 21], [95, 112, 99])
上尉 {'炮兵科教官'}
([87, 89, 91], [91, 88, 90])
西京招待所 {'西安招待

([407, 403, 394], [409, 408, 396])
哲學家 {'科學上的哲學家'}
([109, 107, 106], [120, 117, 110])
最早有關傳染病研究的著作 {'中國最早有關傳染病研究的著作', '為中國最早有關傳染病研究的著作'}
([47, 45, 49], [49, 48, 47])
忠孝橋 {'三重忠孝橋'}
([88, 80, 54], [85, 92, 99])
捷運機場線 {'西區門戶計畫', '「西區門戶計畫」'}
([292, 304, 88], [293, 305, 292])
漢代 {'唐代'}
([180, 351, 71], [354, 182, 353])
伊斯蘭 {'伊斯蘭教'}
([270, 232, 274], [275, 277, 272])
銅鑼灣至太古 {'銅鑼灣至太古一帶'}
([351, 342, 357], [354, 358, 352])
12931ghd {'12931GHD'}
([218, 220, 210], [222, 242, 216])
大陸新移民 {'自1945年大量抵港的大陸新移民', '1945年大量抵港的大陸新移民'}
([244, 243, 254], [250, 252, 270])
新界鄉村的代表 {'由新界鄉村的代表組成'}
([64, 28, 65], [69, 32, 67])
非珠三角地區 {'廣西', '廣西廣府人'}
([14, 76, 28], [15, 17, 77])
廣西 {'粵西'}
([41, 58, 51], [49, 42, 59])
商業方面不是很特出 {'東南亞的大城市'}
([159, 157, 152], [162, 155, 188])
軍閥混戰 {'因為軍閥混戰'}
([62, 267, 144], [64, 269, 146])
福建人 {'閩南人'}
([238, 240, 274], [243, 284, 277])
香港教育水平 {'香港'}
([162, 188, 25], [163, 162, 191])
1980年 {'90年代'}
([91, 89, 87], [92, 100, 91])
傭工 {'外籍家庭傭工'}
([173, 168, 166], [176, 174, 171])
修飾手法 

In [ ]:
x = torch.tensor([0.5,1,0.3])
y = torch.tensor([0.3,0.8,1])
x = x.unsqueeze(0)
y = y.unsqueeze(-1)
z = x*y

z = torch.triu(z, diagonal=1).numpy()
print(z)

In [ ]:
from numpy import unravel_index
unravel_index(z.argmax(), z.shape)


In [ ]:
all_num = 0
match_num =0

for c,q,a in ds.dev:
    all_num = all_num+1
    pred = myfriend.process(c,q)
    if pred == a:
        match_num = match_num +1

print(all_num,match_num,match_num/all_num)
# 3524 2332 0.6617480136208853 

# F1